# Programación para la bioinformática

## Unidad 5: *Testing* y calidad del software

### Instrucciones de uso
A continuación, se presentarán explicaciones y ejemplos de uso de pruebas de código o *software tests*. Recordad que podéis ir ejecutando los ejemplos para obtener sus resultados.


## 1. Introducción

En primer lugar, planteemos las siguientes preguntas:

* **¿Cómo podemos estar seguros de que el código que hemos escrito es correcto?**
* **¿Cómo estamos seguros de que nuestro código es capaz de responder en todos los posibles escenarios?**

Quizá, la respuesta más directa a estas preguntas sea **escribiendo pruebas de código**. Puede parecer extraño que, para probar que efectivamente el código que hemos escrito nos da el resultado correcto y esperado, tengamos que escribir más código que se encargue de verificar el primero. Pero no hacerlo equivale, en el argot, a **caminar sobre la cuerda floja sin red de fondo**.

Las pruebas de código, de aquí en adelante *test*, serán nuestra red, la red que capture los problemas de ejecución de nuestro código y que nos ayude a detectar en una etapa temprana de desarrollo posibles fallos y *bugs*.

Existen diferentes tipos de test: unitarios, de regresión, de punta a punta, de integración... Nosotros nos centraremos en esta unidad en los tests de más bajo nivel, los **unitarios**, que se encargan de comprobar funciones de forma atómica.

Veamos un primer ejemplo:

In [ ]:
from math import sqrt


# Escribimos una función que realiza una suma de forma un tanto especial:

def suma_rara(a, b):
    return pow(sqrt(a), 2) + pow(sqrt(b), 2) 


# Ejecutamos nuestra función utilizando 2 y 3 como argumentos
print(suma_rara(2, 3))

5.0


Como se puede entender al analizar el código, la función calcula la suma de los cuadrados de las raíces de los argumentos, *a* y *b*.

Se trata de una forma un tanto extraña de sumar dos números, pero aparentemente correcta. Ahora bien, **¿qué pasaría si alguno de los argumentos fuera un número negativo?** Comprobémoslo:

In [ ]:
print(suma_rara(-2, 3))

ValueError: ignored

Obtenemos un error de ejecución, ```ValueError```, que nos indica que hay un problema de dominio matemático, y es que la raíz cuadrada de un número negativo no pertenece al dominio de los números reales. En conclusión: nuestra función tiene un error y no es capaz de sumar uno o dos números negativos.

## 2. *Testing*

Como hemos visto en el apartado anterior, necesitamos una forma de comprobar y *testear* nuestro código. Existen diferentes alternativas en Python, incluídas en la librería estándar o librerías externas. Vamos a ver qué opciones nos proporciona Python por defecto:

### 2.1. *Doctest*

Se pueden indicar los propios *test* utilizando *doctest*. *Doctest* pertenece a la librería estándar de Python y es capaz de reconocer porciones de código Python incluídas en la cabecera/documentación de la función y ejecutarlos. Veamos un ejemplo sencillo:

In [ ]:
def square(x):
    """Return the square of x.

    >>> square(2)
    4
    >>> square(-2)
    4
    """
    return x * x


if __name__ == '__main__':
    import doctest
    doctest.testmod()

Como vemos, al ejecutar el anterior código, no aparece ningún mensaje de error. Modifiquemos ahora el código de retorno de la función para comprobar que efectivamente los *test* se están ejecutando:

In [ ]:
def square(x):
    """Return the square of x.

    >>> square(2)
    4
    >>> square(-2)
    4
    """
    return x * 2 * x


if __name__ == '__main__':
    import doctest
    doctest.testmod()

**********************************************************************
File "__main__", line 4, in __main__.square
Failed example:
    square(2)
Expected:
    4
Got:
    8
**********************************************************************
File "__main__", line 6, in __main__.square
Failed example:
    square(-2)
Expected:
    4
Got:
    8
**********************************************************************
1 items had failures:
   2 of   2 in __main__.square
***Test Failed*** 2 failures.


Como podemos observar, ahora se nos avisa en tiempo de ejecución de que ha habido dos problemas al ejecutar el código y de cuáles son los resultados esperados (*Expected*) y obtenidos (*Got*).

*Doctest* es muy útil para indicar a otros programadores cómo debe utilizar nuestras funciones a la vez que se prueba de forma muy simple el propio código, aunque no están indicadas para grandes desarrollos.

Tenéis mucha más información sobre *doctest* en la documentación oficial: [https://docs.python.org/3/library/doctest.html](https://docs.python.org/3/library/doctest.html).

### 2.2. *Unittest*

La librería *unittest* es probablemente la más utilizada por los desarrolladores al no necesitar instalarse de forma externa. Tiene algunas limitaciones en cuanto a cómo descubrir el código que se quiere probar de forma automática, etc., pero es la aproximación por defecto a los *test* unitarios en Python.

Veamos un ejemplo con varias funcionalidades de la librería:

In [ ]:
# Primero de todo importamos la librería:
import unittest


# Definimos una clase para probar algunas funciones de la clase string
# a modo de ejemplo:
class TestStringMethods(unittest.TestCase):

    # Test para probar como pasar un string a mayúsculas:
    def test_upper(self):
        # assertEqual es una función especial que comprueba si ambos 
        # argumentos son iguales o no:
        self.assertEqual('foo'.upper(), 'FOO')

    # Test para probar si un string contiene solo mayúsculas:
    def test_isupper(self):
        # assertTrue comprueba si el valor que se devuelve es TRUE
        self.assertTrue('FOO'.isupper())
        
        # assertFalse hace lo propio para FALSE
        self.assertFalse('Foo'.isupper())

    # Test para comprobar si dividimos un string por espacios de forma 
    # correcta:
    def test_split(self):
        s = 'hello world'
        # Notad que compara entre listas posición por posición:
        self.assertEqual(s.split(), ['hello', 'world'])
        

if __name__ == '__main__':
    """
    Por defecto deberíamos utilizar la siguiente línea:
    
    unittest.main()
    
    pero tenemos que hacerlo de la siguiente forma para que sea 
    posible utilizar unittest en los notebook:
    """
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.009s

OK


Como podéis ver, aparece un mensaje que indica el número de tests que se han ejecutado y su resultado. Si alguno de los tests fuera erróneo, aparecería una *E* y no el carácter "·", y en el caso de que devolvieran un valor erróneo, este sería una *F*.

Si volvemos al ejemplo en el que utilizamos *doctest*, podemos escribir varios *test* de forma diferente:

In [ ]:
import unittest


def square(x):
    return x * x


class TestSquareFunction(unittest.TestCase):

    def test_positive(self):
        a = 2.0
        self.assertEqual(square(a), 4.0)

    def test_negative(self):
        a = -3.0
        self.assertEqual(square(a), 9.0)


if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.011s

OK


### 2.3. Métodos de confirmación

El método *assertEqual* que hemos introducido antes no es el único método de confirmación de la librería *unittest*. Tenéis la lista completa en la [documentación oficial](https://docs.python.org/3/library/unittest.html#assert-methods), aunque aquí tenéis algunos de los más importantes:

* *assertIsNone(x)*: comprueba si la variable *x* es *None*.
* *assertIn(a, b)*: comprueba si *a* está incluído en *b*, muy útil para listas o tuplas.
* *assertNotIn(a, b)*: comprueba la condición contraria a *assertIn*.


### 2.4. Estructura de los *test*

Habitualmente, la estructura que seguiremos para escribir nuestros tests será la siguiente:

```python
def test_if_square_in_list():
    # Preparamos la estructura del test:
    first_squares = [1.0, 4.0, 9.0, 16.0]

    # Ejecutamos la función que queremos probar:
    a_square = square(2.0)

    # Devolvemos el valor del test:
    assert (a_square in first_square)

```

**Es muy importante notar que para que *unittest* descubra los test que queremos ejecutar, la palabra *test* ha de aparecer, ya sea en mayúscula o en cualquier combinación de mayúsculas y minúsculas, al principio del nombre de nuestros *test***.

Otra forma de organizar los *test* es utilizando las funciones ***setUp()*** y ***teardown()*** que se encargan, respectivamente, de preparar la estructura del test y de limpiar, si hace falta, al finalizar el test. Veamos un sencillo ejemplo de ello:

```python
import unittest

class TestPhoneBook(unittest.TestCase):
    """ Clase para probar si ciertos países están en la agenda de prefijos."""
    
    def setUp(self):
        """ Creamos nuestra agenda de prefijos """
        self.phonebook = {34: 'Spain', 33: 'France', 32: 'Belgium', 44: 'UK'}

    def tearDown(self):
        """ Limpiamos la agenda al finalizar el test """
        self.phonebook = None

    def test_if_spain_included(self):
        """ Probamos si España está incluída. """
        self.assertIn('Spain', self.phonebook.values())
                         
```

### 2.5. Resumen

Hemos visto diferentes formas de probar nuestro código y algunos ejemplos muy sencillos. En el *notebook* de la entrega trabajaremos más a fondo algunos de estos conceptos.

## 3. Programación Orientada a Objetos

En unidades anteriores os pedimos documentaros sobre el paradigma de la programación orienta a objetos (POO) y os hemos ido ofreciendo ejemplos de uso, aunque sin entrar en definiciones formales de este tipo de conceptos puesto que estas quedarían fuera del ámbito de conocimiento de un curso introductorio a la programación como es el nuestro.

En cualquier caso y evitando formalismos, la POO es una poderosa herramienta para hacer de nuestro código más modular y reusable. La modularidad es un concepto poderorísimo: nos ayuda a **encapsular funcionalidades similares y esto nos permite diseñar pruebas de código más específicas**. Además, este encapsulamiento nos ayuda a agrupar el código en paquetes o librerías que podemos reutilizar en otros proyectos.

Vamos a hacer un repaso de los conceptos más importantes de la POO:

### 3.1. Clases y objetos

Podemos pensar en una clase como en una **plantilla** para crear objetos (**instancias**) de esa tipo o *clase*.

Un ejemplo clásico de clase es el de `Persona`:

In [ ]:
# Para definir una clase, utilizamos la palabra clave 'class':
class Persona:
    # La forma más básica de clase es aquella que no posee atributos ni métodos.
    # En Python debemos indicar con la palabra clave pass que esta clase no define
    # nada más:
    pass


# Podemos crear un objeto de tipo 'Persona' y asignarlo a una variable 'yo':
yo = Persona()

# Mostremos el tipo de la variable 'yo':
print(f"El tipo de la variable yo es: {type(yo)}")

El tipo de la variable yo es: <class '__main__.Persona'>


El ejemplo anterior, aunque ilustrativo de cómo se crea una clase y un objeto de ese nuevo tipo, no es demasiado útil.

Las clases son útiles para encapsular información y hacer acciones sobre esa misma información. A la información que guardamos en un objeto de una clase determinada se la conoce como **atributo**. A las acciones que se definen en una clase se les conoce como **métodos**. Vamos a ver ejemplos de nuevo en nuestra clase `Persona`:

In [ ]:
class Persona:
    def __init__(self, edad=18):
        # __init__ es un método especial que conocemos como constructor.
        #
        # Esta es la forma de definirlo en Python y el constructor es la forma
        # en la que indicamos cómo crear un nuevo objeto de esta clase.
        # self es la forma de referirse al nuevo objeto que estamos creando mediante
        # __init__ y el primer argumento siempre.
        # A continuación añadimos otro argumento, edad, que es una información que
        # querremos guardar en esta clase. Por defecto en este constructor, la edad 
        # es de 18 años.
        self.edad = edad


yo = Persona()
print(f"Mi edad es de {yo.edad} años")    

Mi edad es de 18 años


Hemos ilustrado cómo almacenar cierta información interesante en un objeto de tipo `Persona`, pero ahora vamos a mostrar cómo interactuar con esta información de forma encapsulada:

In [ ]:
class Persona:
    def __init__(self, nacionalidad, edad=18):
        # Nacionalidad es un nuevo parámetro que no tiene valor por defecto
        # Convertimos a minúscula la nacionalidad que nos pasen como argumento
        self.nacionalidad = nacionalidad.lower()
        self.edad = edad

    def misma_nacionalidad(self, otra_persona):
        # Devuelve verdadero si ambas personas tienen la misma personalidad
        return self.nacionalidad == otra_persona.nacionalidad


# Nuestros físicos favoritos
einstein = Persona("Alemania")
bohr = Persona("Dinamarca")

if einstein.misma_nacionalidad(bohr):
    print("Albert Einstein y Niels Bohr tienen la misma nacionalidad")
else:
    print("Albert Einstein y Niels Bohr no tienen la misma nacionalidad")


Albert Einstein y Niels Bohr no tienen la misma nacionalidad


### 3.2. Sobrecarga de métodos

En el ejemplo anterior, la edad es una información que no hemos utilizado y tampoco parecía demasiado útil. Vamos a utilizar el año de nacimiento en su lugar y, además, mostraremos un concepto muy útil: la sobrecarga de funciones.

Hasta ahora, cuando hemos sumado números en Python hemos utilizado expresiones como la siguiente:

```python
a = 1
b = 2

c = a + b
```

Pero habíamos pasado por alto algo muy importante en Python: **¡todo se considera un objeto, incluso las funciones!**

Los números son objetos de las clases `int` o `float` por ejemplo, y la operación de suma `+` se traduce internamente como la función `__add__(self, other)`. Lo mismo pasa para la resta (`__sub__`) y otras operaciones aritméticas (podéis encontrar una lista [aquí](https://docs.python.org/3/reference/datamodel.html?highlight=__add__#emulating-numeric-types)).

Incluso cuando invocamos la función `print()` sobre un objeto se llama de forma interna al método de esa clase `__str__()` (documentación [link text](https://docs.python.org/3/reference/datamodel.html?highlight=__add__#object.__str__)).

Vamos a combinar todos estos conceptos en otro ejemplo:

In [ ]:
from datetime import date


class Persona:
    def __init__(self, nacionalidad, nacido=1900):
        self.nacionalidad = nacionalidad.lower()
        self.nacido = nacido
        # Calculamos la diferencia entre el año actual y cuándo nació esta persona
        self.edad = abs(date.today().year - self.nacido)

    def misma_nacionalidad(self, otra_persona):
        # Devuelve verdadero si ambas personas tienen la misma personalidad
        return self.nacionalidad == otra_persona.nacionalidad

    def __eq__(self, other):
        # Sobrecarga del método ==
        return self.edad == other.edad

    def __lt__(self, other):
        # Sobrecarga del método <
        return self.edad < other.edad

    def __le__(self, other):
        # Sobrecarga del método <=
        return self.edad <= other.edad

    def __gt__(self, other):
        # Sobrecarga del método >
        return self.edad > other.edad

    def __ge__(self, other):
        # Sobrecarga del método >=
        return self.edad >= other.edad

    def __str__(self):
        # Sobrecarga de representación como string
        return f"Nació en {self.nacionalidad.capitalize()} en el año {self.nacido}"


# Gracias a los diferentes métodos que hemos sobrecargado, ahora podemos comparar
# dos objetos de tipo Persona por su edad, utilizando los operadores >,<, ==, etc.
einstein = Persona("Alemania", 1879)
bohr = Persona("Dinamarca", 1885)

print(f"Albert Einstein nació antes que Niels Bohr? {einstein > bohr}")

print(f"Albert Einstein nació el mismo año que Niels Bohr? {einstein == bohr}")

print("Albert Einstein.", einstein)

Albert Einstein nació antes que Niels Bohr? True
Albert Einstein nació el mismo año que Niels Bohr? False
Albert Einstein. Nació en Alemania en el año 1879


### 3.3. Resumen

Sin entrar en formalidades, hemos visto cómo crear nuevos objetos en Python utilizando el concepto de clases. Hemos visto cómo añadir información relevante para esos objetos vía atributos de clase y hemos aprendido a realizar acciones sobre los atributos, primero utilizando métodos de clase y después sobrecargando métodos.